In [15]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from collections import Counter, defaultdict

In [16]:
data_url = "https://data.heroku.com/dataclips/ouwgbrnpidvmnknflsjhpziwkthe.json?access-token=5b33f8a0-2f7a-4f86-8885-5ebb54df0af5"
#data_url = "https://data.heroku.com/dataclips/ouwgbrnpidvmnknflsjhpziwkthe.json?access-token=5b33f8a0-2f7a-4f86-8885-5ebb54df0af5"
#data_url = "https://data.heroku.com/dataclips/wuvgmaaogmtawpqguturnsewzgoa.json?access-token=9855f9b6-a0b3-4299-92e4-3e480bce357d"
#data_url = "https://data.heroku.com/dataclips/wuvgmaaogmtawpqguturnsewzgoa.json?access-token=9855f9b6-a0b3-4299-92e4-3e480bce357d"

In [17]:
hit_id = '31ANT7FQN89OXUPLFCQGUX3R6WI5HX' # Main run main exp

In [18]:
url = data_url
json_object = requests.get(url).json()
participant_info = []
count = 0
total_bonus = 0
pid = set()
real_participants = {}

time_data = []
bonuses = {}
p_conditions = {}
p_bonuses = {}
worker_ids = []
assignment_ids = []
#print(json_object["values"][0])
for idx_participant, participant_data in enumerate(json_object["values"]):
    #print(participant_data)
    if  (participant_data[4] !="127.0.0.1") and (participant_data[-2] == "live") and (participant_data[-3] == 4) and participant_data[3] == hit_id:
        time_data.append([participant_data[12], participant_data[13]])
        d = json.loads(participant_data[-1])
        json_data = d["data"]
        count += 1
        try:
            #print(idx_participant, len(participant_data[-1]), participant_data[-3],  participant_data[1], participant_data[2],participant_data[3],json_data[0]["trialdata"]["time_elapsed"]/60/60,json.loads(participant_data[-1])["questiondata"]["final_bonus"])
            pid.add(participant_data[2])
            worker_ids.append(participant_data[2])
            assignment_ids.append(participant_data[1])
            x = json.loads(participant_data[-1])["questiondata"]["final_bonus"]
            #print(x)
            bonus = float(x)
            if bonus < 0:
                bonus = 0
            p_bonuses[participant_data[2]] = x
            total_bonus += bonus
            bonuses[participant_data[2]] = bonus
            participant_info.append([idx_participant, len(participant_data[-1]), participant_data[-3],  participant_data[1], participant_data[2],participant_data[3],json_data[0]["trialdata"]["time_elapsed"]/60/60,json.loads(participant_data[-1])["questiondata"]["final_bonus"]])
            participant_df = pd.DataFrame([trial["trialdata"] for trial in json.loads(participant_data[-1])["data"]])
            participant_df["pid"] = idx_participant
            real_participants[participant_data[2]] = participant_df
            p_conditions[participant_data[2]] = d['condition']
        except Exception as e:
            print(e)
#             print(idx_participant,len(participant_data[-1]), participant_data[-3],  participant_data[1],  participant_data[2], participant_data[3],  np.nan, np.nan)
#             participant_info.append([idx_participant,len(participant_data[-1]), participant_data[-3],  participant_data[1],  participant_data[2], participant_data[3],  np.nan, np.nan])
#print(count)
print(total_bonus)
print(len(worker_ids), len(assignment_ids))
#print(real_participants)

230.26000000000005
141 141


In [19]:
print(len(real_participants))

141


In [20]:
num_quizes = []
max_q = 4
print(len(worker_ids))
for i, w in enumerate(worker_ids):
    participant_df = real_participants[w]
    quiz_df = participant_df[participant_df.trial_type == "survey-multi-choice"]
    num_quizes.append(len(quiz_df))
print(num_quizes)
idxs = [i for i, q in enumerate(num_quizes) if q<=max_q]
print(len(idxs)/len(num_quizes))
print(len(idxs))

141
[2, 5, 1, 5, 2, 5, 1, 1, 1, 3, 1, 2, 10, 3, 2, 3, 3, 3, 1, 2, 8, 6, 1, 12, 2, 12, 2, 3, 2, 8, 6, 1, 8, 3, 2, 11, 4, 1, 1, 3, 14, 1, 2, 2, 4, 3, 6, 6, 11, 1, 2, 3, 1, 1, 1, 2, 1, 4, 3, 1, 1, 1, 1, 1, 3, 1, 2, 3, 3, 1, 1, 1, 1, 1, 1, 21, 3, 4, 1, 2, 1, 1, 1, 1, 1, 1, 12, 1, 1, 4, 4, 3, 3, 2, 1, 2, 2, 16, 1, 12, 3, 9, 5, 1, 1, 11, 2, 1, 7, 1, 1, 6, 1, 1, 1, 11, 1, 1, 1, 3, 5, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 28, 5, 1, 5, 1, 9, 7, 2, 2]
0.7730496453900709
109


In [21]:
p_scores = {}
p_click_sequences = {}
p_ground_truths = {}
p_num_clicks = {}
condition_num_clicks = defaultdict(list)
data = defaultdict(lambda: defaultdict())
num_quizes = []
n_p = 0
c = 0
n_cs = []
for i, w in enumerate(worker_ids):
    if i in idxs:
        participant_df = real_participants[w]
        quiz_df = participant_df[participant_df.trial_type == "survey-multi-choice"]
        num_quizes.append(len(quiz_df))
        condition = p_conditions[w]
        temp_df = participant_df[participant_df.trial_type=="mouselab-mdp"]
        scores = temp_df.score.tolist()
        queries = temp_df['queries'].tolist()
        click_sequences = [[int(q) for q in query['click']['state']['target']] + [0] for query in queries]
        num_clicks = [len(click_sequence) for click_sequence in click_sequences]
        click_count = Counter(num_clicks)
        ground_truths = temp_df['stateRewards'].tolist()
        ground_truths = [[0] + gt[1:] for gt in ground_truths]
        n_c = sum([len(cs) for cs in click_sequences[-10:]])
        if n_c == 10: # Disengagement check
            c += 1
            continue
        n_cs.append(n_c-10)
        n_p += 1
        p_scores[i] = scores
        p_ground_truths[i] = ground_truths
        p_click_sequences[i] = click_sequences
        p_num_clicks[i] = [len(click_sequence)-1 for click_sequence in click_sequences]
        data['ground_truths'] = p_ground_truths
        data['clicks'] = p_click_sequences
        data['scores'] = p_scores
        condition_num_clicks[condition].append(p_num_clicks[i])
print(num_quizes, n_p, n_cs, c)

[2, 1, 2, 1, 1, 1, 3, 1, 2, 3, 2, 3, 3, 3, 1, 2, 1, 2, 2, 3, 2, 1, 3, 2, 4, 1, 1, 3, 1, 2, 2, 4, 3, 1, 2, 3, 1, 1, 1, 2, 1, 4, 3, 1, 1, 1, 1, 1, 3, 1, 2, 3, 3, 1, 1, 1, 1, 1, 1, 3, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 3, 3, 2, 1, 2, 2, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2] 92 [13, 329, 43, 63, 141, 10, 138, 284, 120, 54, 88, 8, 135, 304, 4, 229, 130, 19, 65, 112, 326, 112, 14, 88, 81, 84, 1, 88, 47, 108, 94, 5, 50, 218, 328, 329, 2, 329, 133, 99, 324, 80, 219, 23, 89, 95, 9, 213, 329, 327, 166, 100, 95, 2, 128, 33, 1, 7, 28, 110, 56, 156, 68, 117, 3, 5, 63, 7, 328, 49, 97, 82, 110, 60, 111, 105, 95, 62, 22, 72, 40, 64, 109, 72, 1, 61, 88, 73, 62, 73, 90, 168] 17


In [22]:
print(len(data['clicks']))

92
